# Advanced Topic Modeling: Part 1

- MALLET
- Low-level API

In [5]:
import dariah
import cophi

In [2]:
model, _ = dariah.topics(directory="data/british-fiction-corpus",
                         stopwords=100,
                         num_topics=10,
                         num_iterations=1000,
                         implementation="mallet",
                         executable="/home/severin/Downloads/mallet-2.0.8/bin/mallet")

In [10]:
corpus = cophi.corpus(directory="data/british-fiction-corpus",
                      lowercase=True,
                      token_pattern="\p{Letter}+\p{Connector_Punctuation}?\p{Letter}+",
                      metadata=False)

In [13]:
dtm = corpus.dtm.fillna(0).astype(int)
dtm.iloc[:, :5]

,abandon,abandoned,abandoning,abandonment,abandons
fielding_tom,2,15,2,0,0
thackeray_lyndon,0,0,1,0,0
fielding_joseph,3,5,0,0,0
dickens_bleak,3,2,0,0,1
thackeray_vanity,0,10,0,0,0
trollope_prime,11,7,0,2,1
dickens_david,2,6,0,0,0
eliot_adam,1,0,1,0,0
eliot_mill,0,0,0,1,0
trollope_phineas,14,1,0,0,0


In [17]:
stopwords = corpus.mfw(100)
stopwords[:10]

['the', 'and', 'to', 'of', 'in', 'he', 'that', 'was', 'his', 'it']

In [19]:
stopwords = stopword

len(corpus.hapax)

13037